# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 1:54PM,242233,20,ALU0007344,Alumier Labs Inc.,Released
1,Jul 19 2022 1:54PM,242233,20,ALUR065320250,Alumier Labs Inc.,Released
2,Jul 19 2022 1:54PM,242233,20,ALUR091136687,Alumier Labs Inc.,Released
3,Jul 19 2022 1:54PM,242233,20,ALUR198908928,Alumier Labs Inc.,Released
4,Jul 19 2022 1:54PM,242233,20,ALUR308939970,Alumier Labs Inc.,Released
5,Jul 19 2022 1:54PM,242233,20,ALUR438235860,Alumier Labs Inc.,Released
6,Jul 19 2022 1:54PM,242233,20,ALUR441516529,Alumier Labs Inc.,Released
7,Jul 19 2022 1:54PM,242233,20,ALUR712788538,Alumier Labs Inc.,Released
8,Jul 19 2022 1:54PM,242233,20,ALUR714881422,Alumier Labs Inc.,Released
9,Jul 19 2022 1:54PM,242233,20,ALUR732644729,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,242225,Released,1
39,242229,Released,25
40,242230,Released,2
41,242231,Released,1
42,242233,Released,19


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242225,NaN,NaN,1.0
242229,NaN,NaN,25.0
242230,NaN,NaN,2.0
242231,NaN,NaN,1.0
242233,NaN,NaN,19.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242122,0.0,0.0,1.0
242123,19.0,3.0,1.0
242124,0.0,0.0,1.0
242129,0.0,0.0,1.0
242137,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242122,0,0,1
242123,19,3,1
242124,0,0,1
242129,0,0,1
242137,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242122,0,0,1
1,242123,19,3,1
2,242124,0,0,1
3,242129,0,0,1
4,242137,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242122,,,1
1,242123,19,3,1
2,242124,,,1
3,242129,,,1
4,242137,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 1:54PM,242233,20,Alumier Labs Inc.
19,Jul 19 2022 1:46PM,242231,19,ACG North America LLC
20,Jul 19 2022 1:43PM,242230,12,Hush Hush
22,Jul 19 2022 1:35PM,242229,19,"AdvaGen Pharma, Ltd"
47,Jul 19 2022 12:43PM,242225,10,"Amcyte Pharma, Inc."
48,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance
49,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc."
52,Jul 19 2022 11:46AM,242217,18,Eye Pharma Inc
53,Jul 19 2022 11:45AM,242215,10,"Methapharm, Inc."
58,Jul 19 2022 11:40AM,242209,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 1:54PM,242233,20,Alumier Labs Inc.,,,19
1,Jul 19 2022 1:46PM,242231,19,ACG North America LLC,,,1
2,Jul 19 2022 1:43PM,242230,12,Hush Hush,,,2
3,Jul 19 2022 1:35PM,242229,19,"AdvaGen Pharma, Ltd",,,25
4,Jul 19 2022 12:43PM,242225,10,"Amcyte Pharma, Inc.",,,1
5,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,,,1
6,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",,,3
7,Jul 19 2022 11:46AM,242217,18,Eye Pharma Inc,,,1
8,Jul 19 2022 11:45AM,242215,10,"Methapharm, Inc.",,2,3
9,Jul 19 2022 11:40AM,242209,20,"ACI Healthcare USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 1:54PM,242233,20,Alumier Labs Inc.,19,,
1,Jul 19 2022 1:46PM,242231,19,ACG North America LLC,1,,
2,Jul 19 2022 1:43PM,242230,12,Hush Hush,2,,
3,Jul 19 2022 1:35PM,242229,19,"AdvaGen Pharma, Ltd",25,,
4,Jul 19 2022 12:43PM,242225,10,"Amcyte Pharma, Inc.",1,,
5,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,1,,
6,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",3,,
7,Jul 19 2022 11:46AM,242217,18,Eye Pharma Inc,1,,
8,Jul 19 2022 11:45AM,242215,10,"Methapharm, Inc.",3,2,
9,Jul 19 2022 11:40AM,242209,20,"ACI Healthcare USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 1:54PM,242233,20,Alumier Labs Inc.,19,,
1,Jul 19 2022 1:46PM,242231,19,ACG North America LLC,1,,
2,Jul 19 2022 1:43PM,242230,12,Hush Hush,2,,
3,Jul 19 2022 1:35PM,242229,19,"AdvaGen Pharma, Ltd",25,,
4,Jul 19 2022 12:43PM,242225,10,"Amcyte Pharma, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 1:54PM,242233,20,Alumier Labs Inc.,19.0,NaN,NaN
1,Jul 19 2022 1:46PM,242231,19,ACG North America LLC,1.0,NaN,NaN
2,Jul 19 2022 1:43PM,242230,12,Hush Hush,2.0,NaN,NaN
3,Jul 19 2022 1:35PM,242229,19,"AdvaGen Pharma, Ltd",25.0,NaN,NaN
4,Jul 19 2022 12:43PM,242225,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 1:54PM,242233,20,Alumier Labs Inc.,19.0,0.0,0.0
1,Jul 19 2022 1:46PM,242231,19,ACG North America LLC,1.0,0.0,0.0
2,Jul 19 2022 1:43PM,242230,12,Hush Hush,2.0,0.0,0.0
3,Jul 19 2022 1:35PM,242229,19,"AdvaGen Pharma, Ltd",25.0,0.0,0.0
4,Jul 19 2022 12:43PM,242225,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3148637,25.0,40.0,13.0
12,242230,2.0,0.0,0.0
15,484346,5.0,6.0,14.0
18,242217,1.0,0.0,0.0
19,968751,29.0,6.0,19.0
20,1453159,99.0,5.0,0.0
21,1452819,6.0,0.0,0.0
22,484422,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3148637,25.0,40.0,13.0
1,12,242230,2.0,0.0,0.0
2,15,484346,5.0,6.0,14.0
3,18,242217,1.0,0.0,0.0
4,19,968751,29.0,6.0,19.0
5,20,1453159,99.0,5.0,0.0
6,21,1452819,6.0,0.0,0.0
7,22,484422,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,25.0,40.0,13.0
1,12,2.0,0.0,0.0
2,15,5.0,6.0,14.0
3,18,1.0,0.0,0.0
4,19,29.0,6.0,19.0
5,20,99.0,5.0,0.0
6,21,6.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,25.0
1,12,Released,2.0
2,15,Released,5.0
3,18,Released,1.0
4,19,Released,29.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,13.0,0.0,14.0,0.0,19.0,0.0,0.0,0.0
Executing,40.0,0.0,6.0,0.0,6.0,5.0,0.0,0.0
Released,25.0,2.0,5.0,1.0,29.0,99.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,13.0,0.0,14.0,0.0,19.0,0.0,0.0,0.0
1,Executing,40.0,0.0,6.0,0.0,6.0,5.0,0.0,0.0
2,Released,25.0,2.0,5.0,1.0,29.0,99.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,13.0,0.0,14.0,0.0,19.0,0.0,0.0,0.0
1,Executing,40.0,0.0,6.0,0.0,6.0,5.0,0.0,0.0
2,Released,25.0,2.0,5.0,1.0,29.0,99.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()